In [1]:
import boto3 

from boto3.dynamodb.conditions import Key, Attr

import pandas as pd

from pandasgui import show

import numpy as np

from decimal import *

from IPython.display import clear_output

In [2]:
dynamodb = boto3.resource( 'dynamodb' )

In [3]:
adspp_table = dynamodb.Table( 'ads_passenger_processed' )

In [4]:
query_input = dict(
                   IndexName = 'groupMetadataID-index',
    
                   Limit = 1000,
    
                   FilterExpression = Attr( 'topic' ).eq( '/apollo/canbus/chassis' ),

                   KeyConditionExpression = Key( 'groupMetadataID' ).eq( '06cbdbc0-db4d-11ee-a158-97f8443fd730' ),
                  )

In [5]:
query_output = adspp_table.query( **query_input )

In [6]:
query_output_df = pd.DataFrame.from_dict( pd.json_normalize( query_output[ 'Items' ] ), orient = 'columns' )

In [7]:
rows = 1000
while ( 'LastEvaluatedKey' in query_output.keys() ):

    query_input[ 'ExclusiveStartKey' ] = query_output[ 'LastEvaluatedKey' ]

    query_output = adspp_table.query( **query_input )

    temp_df = pd.DataFrame.from_dict( pd.json_normalize( query_output[ 'Items' ] ), orient = 'columns' )

    query_output_df = pd.concat( [ query_output_df, temp_df ] )

    rows = rows + 1000

In [8]:
print( rows )

1384000


In [9]:
chassis_df = query_output_df.copy()

In [10]:
chassis_df = chassis_df.sort_values( 'time' )

In [11]:
query_input = dict(
                   IndexName = 'groupMetadataID-index',
    
                   Limit = 1000,
    
                   FilterExpression = Attr( 'topic' ).eq( '/apollo/sensor/gnss/best_pose' ),

                   KeyConditionExpression = Key( 'groupMetadataID' ).eq( '06cbdbc0-db4d-11ee-a158-97f8443fd730' ),
                  )

In [12]:
query_output = adspp_table.query( **query_input )

In [13]:
query_output_df = pd.DataFrame.from_dict( pd.json_normalize( query_output[ 'Items' ] ), orient = 'columns' )

In [14]:
while ( 'LastEvaluatedKey' in query_output.keys() ):

    query_input[ 'ExclusiveStartKey' ] = query_output[ 'LastEvaluatedKey' ]

    query_output = adspp_table.query( **query_input )

    temp_df = pd.DataFrame.from_dict( pd.json_normalize( query_output[ 'Items' ] ), orient = 'columns' )

    query_output_df = pd.concat( [ query_output_df, temp_df ] )

In [15]:
best_pose_df = query_output_df.copy()

In [16]:
best_pose_df = best_pose_df.sort_values( 'time' )

In [18]:
def GetStateChange( previous, current ):

    if ( current == 'COMPLETE_AUTO_DRIVE' ):

        if ( previous == 'COMPLETE_AUTO_DRIVE' ):

            return 'A->A'

        elif ( previous == 'EMERGENCY_MODE' ) or ( previous == 'COMPLETE_MANUAL' ):

            return 'M->A'

        else:

            return 'U->A'

    elif ( current == 'EMERGENCY_MODE' ) or ( current == 'COMPLETE_MANUAL' ):

        if ( previous == 'COMPLETE_AUTO_DRIVE' ):

            return 'A->M'

        elif ( previous == 'EMERGENCY_MODE' ) or ( previous == 'COMPLETE_MANUAL' ):

            return 'M->M'

        else:

            return 'U->M'

    else:

        if ( previous == 'COMPLETE_AUTO_DRIVE' ):

            return 'A->U'

        elif ( previous == 'EMERGENCY_MODE' ) or ( previous == 'COMPLETE_MANUAL' ):

            return 'M->U'

        else:

            return 'U->U'

In [19]:
modified_chassis_df = chassis_df.copy()

In [20]:
drivingMode_list = modified_chassis_df[ 'drivingMode' ].tolist()

In [21]:
StateChange_list = []
for index in range( len( drivingMode_list ) ):

    if ( index == 0 ):

        previous_driving_mode = 'Unknown'

    else:

        previous_driving_mode = drivingMode_list[ index - 1 ]

    current__driving_mode = drivingMode_list[ index ]

    state_change = GetStateChange( previous_driving_mode, current__driving_mode )

    StateChange_list.append( state_change )

In [22]:
modified_chassis_df[ 'StateChange' ] = StateChange_list

In [23]:
print( modified_chassis_df[ 'StateChange' ].value_counts() )

StateChange
A->A    38460
M->M    14706
M->A       20
A->M       20
U->M        1
Name: count, dtype: int64


In [24]:
def angular_distance( latitude, longitude ): # angular distance from latitude, longitude = 0 on globe 

    return np.arccos( np.cos( latitude ) * np.cos( longitude ) )

In [25]:
modified_best_pose_df = best_pose_df.copy()

In [26]:
latitudeStdDev_list = modified_best_pose_df[ 'latitudeStdDev' ].tolist()

longitudeStdDev_list = modified_best_pose_df[ 'longitudeStdDev' ].tolist()

In [27]:
LatLonStdDev_AngDist_list = []
for latitudeStdDev, longitudeStdDev in zip( latitudeStdDev_list, longitudeStdDev_list ):

    latitudeStdDev = float( latitudeStdDev ) * np.pi / 180

    longitudeStdDev = float( longitudeStdDev ) * np.pi / 180

    latlon_stddev_angdist = angular_distance( latitudeStdDev, longitudeStdDev )

    latlon_stddev_angdist = Decimal( latlon_stddev_angdist * 180 / np.pi )

    LatLonStdDev_AngDist_list.append( latlon_stddev_angdist )

In [28]:
modified_best_pose_df[ 'LatLonStdDev_AngDist' ] = LatLonStdDev_AngDist_list

In [34]:
modified_chassis_pose_df = pd.concat( [ modified_chassis_df, modified_best_pose_df ] )

In [35]:
modified_chassis_pose_df = modified_chassis_pose_df.sort_values( 'time' )

In [38]:
modified_chassis_pose_df['time'] = modified_chassis_pose_df['time'].astype(float)

In [46]:
modified_chassis_pose_df['time'] = modified_chassis_pose_df['time'].astype('datetime64[ms]')

In [47]:
modified_chassis_pose_df

,yawRate,drivingMode,topic,msgsize,time,gearLocation,metadataID,throttlePercentage,steeringPercentage,brakePercentage,...,measurementTime,extendedSolutionStatus,heightStdDev,numSatsMulti,heightMsl,longitude,solutionAge,solStatus,numSatsInSolution,LatLonStdDev_AngDist
30,0,COMPLETE_MANUAL,/apollo/canbus/chassis,134,2023-08-31 15:05:01,GEAR_PARKING,07cb77ce-db4d-11ee-a158-97f8443fd730,0,-34.701275,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,0,COMPLETE_MANUAL,/apollo/canbus/chassis,134,2023-08-31 15:05:01,GEAR_PARKING,07cb77ce-db4d-11ee-a158-97f8443fd730,0,-34.701275,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,0,COMPLETE_MANUAL,/apollo/canbus/chassis,134,2023-08-31 15:05:01,GEAR_PARKING,07cb77ce-db4d-11ee-a158-97f8443fd730,0,-34.701275,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,0,COMPLETE_MANUAL,/apollo/canbus/chassis,134,2023-08-31 15:05:01,GEAR_PARKING,07cb77ce-db4d-11ee-a158-97f8443fd730,0,-34.701275,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,COMPLETE_MANUAL,/apollo/canbus/chassis,134,2023-08-31 15:05:01,GEAR_PARKING,07cb77ce-db4d-11ee-a158-97f8443fd730,0,-34.701275,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,2023-08-31 15:34:17,GEAR_DRIVE,daf57b0c-db59-11ee-a158-97f8443fd730,12.7,0.74436826,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,2023-08-31 15:34:17,GEAR_DRIVE,daf57b0c-db59-11ee-a158-97f8443fd730,13.4,0.87169445,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,2023-08-31 15:34:17,GEAR_DRIVE,daf57b0c-db59-11ee-a158-97f8443fd730,13.8,0.87169445,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,0,EMERGENCY_MODE,/apollo/canbus/chassis,134,2023-08-31 15:34:17,GEAR_DRIVE,daf57b0c-db59-11ee-a158-97f8443fd730,13.8,0.87169445,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
